In [1]:
import pandas as pd

dados = pd.read_csv('./manutencao_preditiva.csv')
dados

,UDI,ID Produto,Tipo,Temperatura Ar [K],Temperatura Processo [K],Velocidade Rotacao [rpm],Torque [Nm],Desgaste Ferramenta [min],Alvo,Tipo da Falha
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure
...,...,...,...,...,...,...,...,...,...,...
9995,9996,M24855,M,298.8,308.4,1604,29.5,14,0,No Failure
9996,9997,H39410,H,298.9,308.4,1632,31.8,17,0,No Failure
9997,9998,M24857,M,299.0,308.6,1645,33.4,22,0,No Failure
9998,9999,H39412,H,299.0,308.7,1408,48.5,25,0,No Failure


In [2]:
def padronizar_tipo(valor):
    if valor == 'L':
        return 0.0
    elif valor == 'M':
        return 0.5
    elif valor == 'H':
        return 1.0
    else:
        return ValueError

In [3]:
map_tipo = {
    "L":1,
    "M":2,
    "H":0,
}

In [4]:
def normalziar_coluna(var1):
    normalizado = []
    for i in range(len(var1)):
        normalizado.append((var1[i] - min(var1)) / (max(var1) - min(var1)))
    return normalizado

In [5]:
def delta_entre_coluna(var1, var2):
    diferenca = []
    if len(var1) == len(var2):
        for i in range(len(var1)):
            diferenca.append( (abs(var1[i] - var2[i])) )
        return diferenca
    else:
        return ValueError
        

In [6]:
def media_da_coluna(x):
    return (sum(x)/len(x))

In [7]:
def padronizar_falhas(valor):
    if valor == 'No Failure':
        return 0
    if valor == 'Heat Dissipation Failure':
        return 1
    elif valor == 'Power Failure':
        return 10
    elif valor == 'Overstrain Failure':
        return 100
    elif valor == 'Tool Wear Failure':
        return 1000
    elif valor == 'Random Failures':
        return 10000
    else:
        return ValueError

In [8]:
map_falhas = {
    'No Failure': 0,
    'Heat Dissipation Failure':1,
    'Power Failure': 2,
    'Overstrain Failure': 3,
    'Tool Wear Failure': 4,
    'Random Failures': 5,
}

In [9]:
dados['Tipo'].value_counts()

Tipo
L    6000
M    2997
H    1003
Name: count, dtype: int64

In [10]:
falhas = dados['Tipo da Falha'].value_counts()
falhas

Tipo da Falha
No Failure                  9652
Heat Dissipation Failure     112
Power Failure                 95
Overstrain Failure            78
Tool Wear Failure             45
Random Failures               18
Name: count, dtype: int64

In [11]:
int(falhas['No Failure'])

9652

In [12]:
dados['Alvo'].value_counts()

Alvo
0    9661
1     339
Name: count, dtype: int64

In [13]:
dados['Falhas Padronizadas'] = dados['Tipo da Falha'].map(map_falhas)

In [14]:
dados['Temp Ar Normalizado'] = normalziar_coluna(dados['Temperatura Ar [K]'])
dados['Temp Processo Normalizado'] = normalziar_coluna(dados['Temperatura Processo [K]'])

In [15]:
dados['RPM Normalizado'] = normalziar_coluna(dados['Velocidade Rotacao [rpm]'])
dados['Torque Normalizado'] = normalziar_coluna(dados['Torque [Nm]'])
dados['Desgaste Normalizado'] = normalziar_coluna(dados['Desgaste Ferramenta [min]'])

In [16]:
dados['Tipo padronizado'] = dados['Tipo'].map(map_tipo)


In [17]:
dados['Delta Temp'] = delta_entre_coluna(dados['Temperatura Processo [K]'], dados['Temperatura Ar [K]'])
dados['Delta Temp Normalizado'] = normalziar_coluna(dados['Delta Temp'])


In [18]:
dados['Delta RPM-Torque Norm'] = delta_entre_coluna(dados['RPM Normalizado'], dados['Torque Normalizado'])

In [19]:
nova_ordem = ['Tipo', 'Tipo padronizado', 'Temperatura Ar [K]', 'Temp Ar Normalizado', 'Temperatura Processo [K]', 'Temp Processo Normalizado', 'Delta Temp', 'Delta Temp Normalizado', 'Velocidade Rotacao [rpm]', 'RPM Normalizado', 'Torque [Nm]', 'Torque Normalizado', 'Delta RPM-Torque Norm', 'Desgaste Ferramenta [min]', 'Desgaste Normalizado', 'Falhas Padronizadas','Tipo da Falha']

In [20]:
dados_normalizado = ['Tipo padronizado', 'Temp Ar Normalizado', 'Temp Processo Normalizado', 'Delta Temp Normalizado','RPM Normalizado', 'Torque Normalizado', 'Delta RPM-Torque Norm', 'Falhas Padronizadas']

In [21]:
sort_order = ['Tipo padronizado', 'Falhas Padronizadas', 'Delta Temp Normalizado', 'Delta RPM-Torque Norm']
sort_ascending = [True, True, True, True]

In [22]:
dados = dados[nova_ordem]
dadosNorm = dados[dados_normalizado]
dados = dados.sort_values(by=sort_order, ascending=sort_ascending)
dadosNorm = dadosNorm.sort_values(by=sort_order, ascending=sort_ascending)
dados

,Tipo,Tipo padronizado,Temperatura Ar [K],Temp Ar Normalizado,Temperatura Processo [K],Temp Processo Normalizado,Delta Temp,Delta Temp Normalizado,Velocidade Rotacao [rpm],RPM Normalizado,Torque [Nm],Torque Normalizado,Delta RPM-Torque Norm,Desgaste Ferramenta [min],Desgaste Normalizado,Falhas Padronizadas,Tipo da Falha
4420,H,0,302.6,0.793478,310.2,0.555556,7.6,0.000000,1873,0.410361,21.3,0.240385,0.169976,87,0.343874,0,No Failure
4499,H,0,302.6,0.793478,310.3,0.567901,7.7,0.022222,1721,0.321886,27.4,0.324176,0.002290,79,0.312253,0,No Failure
4455,H,0,302.4,0.771739,310.1,0.543210,7.7,0.022222,1597,0.249709,28.9,0.344780,0.095071,179,0.707510,0,No Failure
4494,H,0,302.6,0.793478,310.4,0.580247,7.8,0.044444,1649,0.279977,30.1,0.361264,0.081287,62,0.245059,0,No Failure
4498,H,0,302.6,0.793478,310.4,0.580247,7.8,0.044444,1614,0.259604,36.0,0.442308,0.182704,74,0.292490,0,No Failure
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7849,M,2,300.3,0.543478,311.7,0.740741,11.4,0.844444,1374,0.119907,47.9,0.605769,0.485862,222,0.877470,4,Tool Wear Failure
8111,M,2,300.4,0.554348,311.8,0.753086,11.4,0.844444,1553,0.224098,33.1,0.402473,0.178375,209,0.826087,4,Tool Wear Failure
8846,M,2,297.4,0.228261,308.8,0.382716,11.4,0.844444,1325,0.091385,62.4,0.804945,0.713560,204,0.806324,4,Tool Wear Failure
3065,M,2,300.1,0.521739,309.2,0.432099,9.1,0.333333,1687,0.302095,27.7,0.328297,0.026201,95,0.375494,5,Random Failures


In [23]:
dadosNorm

,Tipo padronizado,Temp Ar Normalizado,Temp Processo Normalizado,Delta Temp Normalizado,RPM Normalizado,Torque Normalizado,Delta RPM-Torque Norm,Falhas Padronizadas
4420,0,0.793478,0.555556,0.000000,0.410361,0.240385,0.169976,0
4499,0,0.793478,0.567901,0.022222,0.321886,0.324176,0.002290,0
4455,0,0.771739,0.543210,0.022222,0.249709,0.344780,0.095071,0
4494,0,0.793478,0.580247,0.044444,0.279977,0.361264,0.081287,0
4498,0,0.793478,0.580247,0.044444,0.259604,0.442308,0.182704,0
...,...,...,...,...,...,...,...,...
7849,2,0.543478,0.740741,0.844444,0.119907,0.605769,0.485862,4
8111,2,0.554348,0.753086,0.844444,0.224098,0.402473,0.178375,4
8846,2,0.228261,0.382716,0.844444,0.091385,0.804945,0.713560,4
3065,2,0.521739,0.432099,0.333333,0.302095,0.328297,0.026201,5


In [24]:
colunas = dados.columns

print("Valores originais:")
for col in colunas[3:8]:
    print(f"Media da coluna - {col} é: {media_da_coluna(dados[col]):.02f}")

print("-------------------\nValores normalizados: ")
for col in colunas[10:14]:
    print(f"Media da coluna - {col} é: {media_da_coluna(dados[col]):.02f}")

print("-------------------\nValores de Deltas: ")
for col in colunas[14:16]:
    print(f"Media da coluna - {col} é: {media_da_coluna(dados[col]):.02f}")

Valores originais:
Media da coluna - Temp Ar Normalizado é: 0.51
Media da coluna - Temperatura Processo [K] é: 310.01
Media da coluna - Temp Processo Normalizado é: 0.53
Media da coluna - Delta Temp é: 10.00
Media da coluna - Delta Temp Normalizado é: 0.53
-------------------
Valores normalizados: 
Media da coluna - Torque [Nm] é: 39.99
Media da coluna - Torque Normalizado é: 0.50
Media da coluna - Delta RPM-Torque Norm é: 0.32
Media da coluna - Desgaste Ferramenta [min] é: 107.95
-------------------
Valores de Deltas: 
Media da coluna - Desgaste Normalizado é: 0.43
Media da coluna - Falhas Padronizadas é: 0.08


## 70% a 80% para treinar

## 30% a 20% para teste

In [25]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# df_num = dados.select_dtypes(include=['int64','float64'])

# corr_matrix = df_num.corr(method='pearson')

# plt.figure(figsize=(6,5))
# sns.heatmap(corr_matrix, annot=True, fmt='.2f',cmap='coolwarm',cbar=True, square=True)

# plt.title('Matriz de Correlação de Pearson')
# plt.show()

In [26]:
entradas = dadosNorm[['Tipo padronizado',
                      'Temp Ar Normalizado',
                      'Temp Processo Normalizado',
                      'Delta Temp Normalizado',
                      'RPM Normalizado',
                      'Torque Normalizado',
                      'Delta RPM-Torque Norm'
                      ]]

saida = dadosNorm[['Falhas Padronizadas']]

In [27]:
from sklearn.model_selection import train_test_split

xtreino, xteste, ytreino, yteste, = train_test_split(entradas,saida,test_size=0.3)
xtreino


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(100), max_iter=100, activation='relu', random_state=42, verbose=True)

# n_iter_no_change = 50

# Treinar o modelo utilizando os dados de treino

treino = mlp.fit(xtreino, ytreino)
treino

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

y_pred = mlp.predict(xteste)
acuracia = accuracy_score(yteste, y_pred)
matriz_confusao = confusion_matrix(yteste, y_pred)

print(acuracia)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

#Matriz de confusão
cm = confusion_matrix(yteste, y_pred)

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(ytreino), yticklabels=np.unique(ytreino))

1 - 10 neuronios - acuracia = ?

2 - 20 neuronios - acuracia = ?

3 - 100 neuronios - acuracia = ?

4 - (10,10) neuronios - acuracia = ?

5 - (20,20) neuronios - acuracia = ?

6 - (100,100) neuronios - acuracia = ?